In [1]:
%matplotlib inline
from IPython import display
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import *
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
#Setting some inital parameters
height, width = 224, 224
batch_size=64

In [3]:
data_dir = 'C:/pythonn/breast_cancer_data/'
batch_size = 32

def image_generator(height, width):
    datagen = ImageDataGenerator(
        rescale=1./255.,
        validation_split=0.2
    )

    train_ds = datagen.flow_from_directory(
        data_dir,
        batch_size=batch_size,
        subset="training",
        shuffle=True,
        class_mode='binary',
        target_size=(height, width),
        classes={'BENIGN': 0., 'MALIGNANT': 1.}
    )

    val_ds = datagen.flow_from_directory(
        data_dir,
        subset="validation",
        class_mode='binary',
        target_size=(height, width),
        batch_size=batch_size,
        classes={'BENIGN': 0., 'MALIGNANT': 1.}
    )

    return train_ds, val_ds

train_ds, val_ds = image_generator(height, width)

total_image = np.concatenate([train_ds.labels, val_ds.labels])
print('\n\n', {
    'breast_tumor_benign': len(np.where(total_image == 0)[0]),
    'breast_tumor_malignant': len(np.where(total_image == 1)[0])
})

Found 849 images belonging to 2 classes.
Found 211 images belonging to 2 classes.


 {'breast_tumor_benign': 566, 'breast_tumor_malignant': 494}


In [4]:
from tensorflow.keras.layers.experimental import preprocessing
tf.keras.backend.clear_session()
input_shape = (height, width, 3)
base_model = tf.keras.applications.vgg19.VGG19(
    weights='imagenet', 
    include_top=False,
    input_shape=input_shape
)
base_model.trainable = False
model_vgg19 = tf.keras.Sequential()
model_vgg19.add(base_model)
model_vgg19.add(tf.keras.layers.Flatten())
model_vgg19.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_vgg19.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=['acc'])
model_vgg19.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model/vgg19_best.h5', monitor='acc', 
                                                verbose=1, mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="acc", mode="max",restore_best_weights=True, 
                                         patience=5)
callbacks_list = [checkpoint,early]
history = model_vgg19.fit(
        train_ds,
        validation_data=val_ds,
        epochs=5, 
        shuffle=True, 
        verbose=True,
        callbacks=callbacks_list)

Epoch 1/5
15/27 [===============>..............] - ETA: 1:31 - loss: 8.1741 - acc: 0.4839

In [ ]:
train_result = model_vgg19.evaluate(train_ds)
val_result = model_vgg19.evaluate(val_ds)
no_augmented_df = pd.DataFrame(zip(train_result,val_result),
                               columns=['Train','Val'],index=['Loss','Acc'])
no_augmented_df

In [ ]:
# plot learning curve
def plot_learning_curve(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, label='training acc')
    plt.plot(epochs, val_acc, label='validation acc')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy-%')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, label='training loss')
    plt.plot(epochs, val_loss, label='validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

plot_learning_curve(history)